In [1]:
import pandas as pd
import numpy as np
import geojson

# Loading

In [2]:
df = pd.read_csv('data/lista-de-especies-ameacas-2020.csv', sep=';', engine='python')

# Cleaning

In [3]:
# Filtering only the fauna
df = df.loc[df['Fauna/Flora'] == 'Fauna']
# Filtering only the fishes and related 
df = df.loc[df['Grupo'].isin(['Peixes Marinhos', 'Invertebrados Aquáticos', 'Peixes Continentais'])]
# Exploding the state_of_occurence
df = pd.merge(df.reset_index(), df['Estados de Ocorrência'].str.split(';').explode().reset_index(), on='index')
# Stripping the data
df['Estados de Ocorrência_y'] = df['Estados de Ocorrência_y'].str.strip()

# Translating

In [4]:
df.columns = ['index', 'fauna', 'group', 'family', 'simplified_species',
                'commum_name', 'threat_category', 'acronym_threat_category',
                'biome', 'main_threats', 'presence_in_protected_areas',
                'national_action_plan_for_conservation_',
                'fishing_planning', 'level_of_protection_in_the_national_strategy',
                'exclusive_species_of_brazil', 'occurrence_states_x',
                'occurrence_states_y']

In [5]:
columns_translate = ['commum_name','group', 'threat_category', 'biome', 
                        'presence_in_protected_areas', 'national_action_plan_for_conservation_',
                        'fishing_planning', 'exclusive_species_of_brazil']

In [6]:
df = pd.merge(df.reset_index(), df['main_threats'].str.split(';').explode().reset_index(), on='index').drop(columns = 'index')
df = pd.merge(df.reset_index(), df['biome'].str.split(';').explode().reset_index(), on='index').drop(columns = ['index', 'level_0', 'biome_x']).rename(columns={'biome_y':'biome'})

In [7]:
df

,fauna,group,family,simplified_species,commum_name,threat_category,acronym_threat_category,main_threats_x,presence_in_protected_areas,national_action_plan_for_conservation_,fishing_planning,level_of_protection_in_the_national_strategy,exclusive_species_of_brazil,occurrence_states_x,occurrence_states_y,main_threats_y,biome
0,Fauna,Peixes Marinhos,Achiridae,Achirus mucuri,Solha,Vulnerável (VU),VU,Extracao Direta:Caça/Pesca; Outras Atividades ...,Sim,Não,Não,1.0,Sim,BA,BA,Agropecuária,Mata Atlântica
1,Fauna,Peixes Marinhos,Achiridae,Achirus mucuri,Solha,Vulnerável (VU),VU,Extracao Direta:Caça/Pesca; Outras Atividades ...,Sim,Não,Não,1.0,Sim,BA,BA,Poluição,Mata Atlântica
2,Fauna,Peixes Marinhos,Achiridae,Achirus mucuri,Solha,Vulnerável (VU),VU,Extracao Direta:Caça/Pesca; Outras Atividades ...,Sim,Não,Não,1.0,Sim,BA,BA,Queimadas,Mata Atlântica
3,Fauna,Invertebrados Aquáticos,Aeglidae,Aegla brevipalma,Egla,Criticamente em Perigo (CR),CR,Agropecuária; Poluição,Não,Sim,Não,1.0,Sim,SC,SC,Extracao Direta:Caça/Pesca,Mata Atlântica
4,Fauna,Invertebrados Aquáticos,Aeglidae,Aegla camargoi,Egla,Em Perigo (EN),EN,Agropecuária; Outras Atividades Econômicas:Ene...,Não,Sim,Não,1.0,Sim,RS; SC,RS,Extracao Direta:Caça/Pesca,Mata Atlântica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Fauna,Peixes Continentais,Loricariidae,Hypancistrus zebra,Cascudo-zebra-imperial,Criticamente em Perigo (CR),CR,Extracao Direta:Caça/Pesca; Outras Atividades ...,Não,Sim,Sim,2.0,Sim,PA,PA,Assentamento Humano:cidades,Amazônia
1074,Fauna,Peixes Continentais,Loricariidae,Hypancistrus zebra,Cascudo-zebra-imperial,Criticamente em Perigo (CR),CR,Extracao Direta:Caça/Pesca; Outras Atividades ...,Não,Sim,Sim,2.0,Sim,PA,PA,Mineracao,Amazônia
1075,Fauna,Peixes Continentais,Loricariidae,Hypancistrus zebra,Cascudo-zebra-imperial,Criticamente em Perigo (CR),CR,Extracao Direta:Caça/Pesca; Outras Atividades ...,Não,Sim,Sim,2.0,Sim,PA,PA,Poluição,Amazônia
1076,Fauna,Peixes Continentais,Characidae,Hyphessobrycon coelestinus,Desconhecido,Em Perigo (EN),EN,Assentamento Humano:cidades,Sim,Sim,Sim,2.0,Sim,DF; GO,DF,Extracao Direta:Caça/Pesca,Cerrado


In [8]:
translation_dict = {
    'Solha': 'Halibut', 'Egla': 'Egla', 'Bagre': 'Catfish', 'Tubarão-raposa': 'Fox Shark',
    'Peixe-anual': 'Annual Fish', 'Cascudinho-de-caverna': 'Cave Pygmy Suckerfish',
    'Cascudo-cego': 'Blind Suckerfish', 'Desconhecido': 'Unknown', 'Peixe-rei': 'Kingfish',
    'Canivete': 'Penknife Fish', 'Lambari': 'Minnow', 'Sarapó': 'Sarapó', 'Estrela-do-mar': 'Starfish',
    'Lambari, Piaba': 'Minnow, Piaba', 'Lambari ': 'Minnow ', 'Raia-chita': 'Leopard Stingray',
    'Bagre ': 'Catfish ', 'Acari': 'Acari', 'Tuvira-da-juréia': 'Juréia Toothpick Fish',
    'Piabinha': 'Little Minnow', 'Matrinxã, Piabanha ': 'Matrinxã, Piabanha ', 'Piabanha': 'Piabanha',
    'Pirapitinga': 'Pirapitinga', 'Piracanjuba': 'Piracanjuba', 'Vermelha': 'Red Fish',
    'Tubarão-das-galápagos': 'Galapagos Shark', 'Tubarão-galha-branca': 'Whitetip Reef Shark',
    'Cação-fidalgo': 'Cobbler Shark', 'Tubarão-dos-recifes': 'Reef Shark',
    'Tubarão-galhudo': 'Bigeye Thresher Shark', 'Cação-azeiteiro': 'Oil Shark',
    'Cação-noturno': 'Night Shark', 'Cação-mangona': 'Mangona Shark',
    'Tubarão-branco': 'Great White Shark', 'Guaiamum': 'Swimming Crab', 'Ouriço-do-mar': 'Sea Urchin',
    'Peixe-lombriga-listrado': 'Striped Wormfish', 'Tubarão-peregrino': 'Wandering Tattler',
    'Canivete ': 'Penknife ', 'Bagrinho': 'Corydoras', 'Canário-do-mar': 'Sea Canary',
    'Barrigudinho ': 'Big Belly ', 'Guarú-de-apiaí': 'Guarú-de-apiaí', 'Barrigudinho': 'Barrigudinho',
    'Anêmona-gigante': 'Giant Anemone', 'Pirá-tamanduá': 'Giant Armadillo',
    'Piquira-de-duas-listras': 'Two-lined Piquira', 'Cascudinho': 'Suckerfish', 'Esponja': 'Sponge',
    'Joana': 'Joana', 'Jacundá': 'Jacundá', 'Joaninha': 'Joaninha', 'Sabãozinho': 'Soapfish',
    'Peixe-da-chuva': 'Rain Fish', 'Raia-prego-de-cauda-áspera': 'Thorny Stingray', 'Raia': 'Stingray',
    'Cascudo': 'Suckerfish', 'Marisco-do-junco': 'Marsh Mussel', 'Ituí': 'Ituí', 'Neon': 'Neon Tetra',
    'Mero': 'Grouper', 'Garoupa-verdadeira': 'True Grouper', 'Garoupa ': 'Grouper ', 'Búzio-de-chapéu': 'Little Bishop Hat',
    'Vieira': 'Vieira', 'Cação-bico-doce ': 'Smooth Dogfish ', 'Bagre-branco': 'Whitefish',
    'Bagre-marinho': 'Marine Catfish', 'Tubarão-lixa': 'Cave Bottlenose', 'Bagrinho-de-caverna': 'Acará Cachimbo ',
    'Acará': 'Botinho, Reco-reco ', 'Raia-manteiga': 'Piririca Suckerfish', 'Bodião-fogueira': 'Andirá',
    'Acari-cachimbo ': 'Bagrinho-do-tietê', 'Botinho, Reco-reco ': 'Cavalo-marinho', 'Cascudo-piririca': 'Lambari-listrado',
    'Andirá': 'Bodó-seda', 'Bagrinho-do-tietê': 'Imperial Zebra Pleco', 'Cavalo-marinho': 'Red Tetra',
    'Lambari-listrado': 'Timburé', 'Bodó-seda': 'Black Grouper ', 'Cascudo-zebra-imperial': 'True Grouper',
    'Tetra-vermelho': 'Grouper ', 'Timburé': 'Suckerfish ', 'Cherne-negro ': 'Quati Shark',
    'Cherne-verdadeiro': 'Yellow Crab', 'Cascudo ': 'White Marlin', 'Cação-quati': 'Peracuca',
    'Caranguejo-amarelo': 'Aracu', 'Agulhão-branco': 'Minhoca-do-ribeira Cambeva ', 'Peracuca': 'Pacu',
    'Aracu': 'Potato Fish', 'Cambeva-minhoca-do-ribeira ': 'Caranha', 'pacamã': 'Snapper',
    'Peixe-batata': 'Freshwater Snail', 'Caranha': 'Lilac Urchin', 'Pargo': 'Blue Marlin',
    'Caramujo-de-água-doce': 'Manta Ray', 'Ouriço-lilás': 'Bourbon', 'Marlin-azul': 'Minnow',
    'Raia-manta': 'Blue Damsel ', 'Amaripim': 'Striped Minnow', 'Piaba': 'Brain Coral',
    'Donzela-azul ': 'Sailfin', 'Lambari-azul-listrado': 'Old Mouth',
    'Coral-cérebro-da-bahia': 'Striped Shark', 'Coral-vela': 'Spotted Roundnose Shark',
    'Boca-de-velha': 'Speckled Guitarfish ', 'Cação-listrado': 'Pacu', 'Tubarão-bico-doce-pintado': 'Dwarf Suckerfish',
    'Faquinha-arredondada': 'Ray', 'Sirigado ': 'Marbled Electric Ray', 'Badejo-amarelo': 'Electric Ray',
    'Pacu, Pacu-dente-seco, Pacu de cachoeira, Curupeté, Pacu borracha': 'Parasitic Catfish',
    'Pacu-prata ': 'Blind Cave Catfish', 'Raia-amarela': 'Cambeva', 'Raia-sapo': 'Minhoca-do-tietê Cambeva',
    'Peixe-bruxa': 'Cambeva'
}


In [9]:
mapping_dict = {
    'Extracao Direta:Caça/Pesca': 'Fishing',
    ' Outras Atividades Econômicas:Energia': 'Energy production',
    ' Poluição Assoreamento': 'Pollution',
    'Agropecuária': 'Agriculture',
    ' Poluição': 'Pollution',
    'Assentamento Humano:cidades': 'Human Settlement',
    ' Outras Atividades Econômicas:Industrias': 'Industry',
    ' Outras Atividades Econômicas:Transporte': 'Transport',
    ' Queimadas': 'Wildfires',
    'Especies Exoticas': 'Exotic Species',
    ' Mineracao': 'Mining',
    ' Outras Atividades Econômicas:Turismo': 'Tourism',
    'Poluição': 'Pollution',
    ' Assentamento Humano:cidades': 'Human Settlement',
    'Assentamento Humano:rurais': 'Human Settlement',
    ' Especies Exoticas': 'Exotic Species',
    'Outras Atividades Econômicas:Energia': 'Energy production',
    'Mineracao': 'Mining',
    'Desastres Naturais': 'Natural Disasters',
    ' Extracao Direta:Caça/Pesca': 'Fishing',
    'Outras Atividades Econômicas: Barrages Abastecimento/Drenagem': 'Damming/Drainage',
    'Agropecuária Aquacultura': 'Aquaculture Agriculture',
    'Extracao Direta:Remocao De Corais': 'Coral Removal',
    ' Outras Atividades Econômicas: Barrages Abastecimento/Drenagem': 'Damming/Drainage',
    'Outras Atividades Econômicas:Turismo': 'Tourism',
    'Outras Atividades Econômicas:Transporte': 'Transport',
    ' Extracao Direta:Remocao De Corais': 'Coral Removal',
    'Mudanca Na Dinamica Das Especies Nativas': 'Changes in Native Species Dynamics',
    ' Desastres Naturais': 'Natural Disasters',
    ' Mudanca Na Dinamica Das Especies Nativas': 'Changes in Native Species Dynamics',
    'Ameaca Desconhecida': 'Unknown Threat',
    'Extracao Florestal': 'Forest Extraction',
    ' Mudanca Na Dinamica Das Especies Nativas: Patogenias/Parasitas': 'Pathogens/Parasites',
    ' Extracao Florestal': 'Forest Extraction'
}
df['main_threats_y'] = df['main_threats_y'].replace(mapping_dict)

In [10]:
# List of Conservation Status
conservation_status = [
    'Vulnerable (VU)', 'Critically Endangered (CR)', 'Endangered (EN)',
    'Critically Endangered (CR)/Possibly Extinct (PEX)'
]

# Translated Conservation Status
english_conservation_status = [
    'Vulnerable', 'Critically Endangered', 'Endangered',
    'Critically Endangered/Possibly Extinct'
]

# List of Captive Breeding Programs
captive_breeding_programs = ['Yes', 'No']

# Translated Captive Breeding Programs
english_captive_breeding_programs = ['Yes', 'No']

# List of Introduced Species
introduced_species = ['No', 'Yes']

# Translated Introduced Species
english_introduced_species = ['No', 'Yes']

# List of Invasive Species
invasive_species = ['No', 'Yes']

# Translated Invasive Species
english_invasive_species = ['No', 'Yes']

# Conservation Status
conservation_status_dict = dict(zip(conservation_status, english_conservation_status))

# Habitats
# Habitats
habitats_dict = {
    'Mata Atlântica': 'Atlantic Forest',
    ' Pampa': 'Pampa',
    'Pampa': 'Pampa',
    'Amazônia': 'Amazon',
    ' Cerrado': 'Cerrado',
    'Marinho': 'Marine',
    'Caatinga': 'Caatinga',
    'Cerrado': 'Cerrado',
    ' Mata Atlântica': 'Atlantic Forest'# Handling 'nan' value, set to None
}


# Captive Breeding Programs
captive_breeding_programs_dict = dict(zip(captive_breeding_programs, english_captive_breeding_programs))

# Introduced Species
introduced_species_dict = dict(zip(introduced_species, english_introduced_species))

# Invasive Species
invasive_species_dict = dict(zip(invasive_species, english_invasive_species))

conservation_status_dict = {
    'Vulnerável (VU)': 'Vulnerable (VU)',
    'Criticamente em Perigo (CR)': 'Critically Endangered (CR)',
    'Em Perigo (EN)': 'Endangered (EN)',
    'Criticamente em Perigo (CR)/Possivelmente Extinto (PEX)': 'Critically Endangered (CR)/Possibly Extinct (PEX)'
}

# Habitats
cat_species = {
    'Peixes Marinhos': 'Marine Fish',
    'Invertebrados Aquáticos': 'Aquatic Invertebrates',
    'Peixes Continentais': 'Freshwater Fish',
    'Sim':'Yes', 'Não':'No'}



final_translation_dict = {
    **translation_dict,
    **conservation_status_dict,
    **habitats_dict,
    **captive_breeding_programs_dict,
    **introduced_species_dict,
    **invasive_species_dict,
    **cat_species
}

print(final_translation_dict)

{'Solha': 'Halibut', 'Egla': 'Egla', 'Bagre': 'Catfish', 'Tubarão-raposa': 'Fox Shark', 'Peixe-anual': 'Annual Fish', 'Cascudinho-de-caverna': 'Cave Pygmy Suckerfish', 'Cascudo-cego': 'Blind Suckerfish', 'Desconhecido': 'Unknown', 'Peixe-rei': 'Kingfish', 'Canivete': 'Penknife Fish', 'Lambari': 'Minnow', 'Sarapó': 'Sarapó', 'Estrela-do-mar': 'Starfish', 'Lambari, Piaba': 'Minnow, Piaba', 'Lambari ': 'Minnow ', 'Raia-chita': 'Leopard Stingray', 'Bagre ': 'Catfish ', 'Acari': 'Acari', 'Tuvira-da-juréia': 'Juréia Toothpick Fish', 'Piabinha': 'Little Minnow', 'Matrinxã, Piabanha ': 'Matrinxã, Piabanha ', 'Piabanha': 'Piabanha', 'Pirapitinga': 'Pirapitinga', 'Piracanjuba': 'Piracanjuba', 'Vermelha': 'Red Fish', 'Tubarão-das-galápagos': 'Galapagos Shark', 'Tubarão-galha-branca': 'Whitetip Reef Shark', 'Cação-fidalgo': 'Cobbler Shark', 'Tubarão-dos-recifes': 'Reef Shark', 'Tubarão-galhudo': 'Bigeye Thresher Shark', 'Cação-azeiteiro': 'Oil Shark', 'Cação-noturno': 'Night Shark', 'Cação-mangona

In [11]:
unique_values_list =[]
for i in columns_translate:
    df[i] = df[i].replace(final_translation_dict)

In [12]:
df['threat_category'].unique()

array(['Vulnerable (VU)', 'Critically Endangered (CR)', 'Endangered (EN)',
       'Critically Endangered (CR)/Possibly Extinct (PEX)'], dtype=object)

In [13]:
df['group'].unique()

array(['Marine Fish', 'Aquatic Invertebrates', 'Freshwater Fish'],
      dtype=object)

In [14]:
df

,fauna,group,family,simplified_species,commum_name,threat_category,acronym_threat_category,main_threats_x,presence_in_protected_areas,national_action_plan_for_conservation_,fishing_planning,level_of_protection_in_the_national_strategy,exclusive_species_of_brazil,occurrence_states_x,occurrence_states_y,main_threats_y,biome
0,Fauna,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,Extracao Direta:Caça/Pesca; Outras Atividades ...,Yes,No,No,1.0,Yes,BA,BA,Agriculture,Atlantic Forest
1,Fauna,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,Extracao Direta:Caça/Pesca; Outras Atividades ...,Yes,No,No,1.0,Yes,BA,BA,Pollution,Atlantic Forest
2,Fauna,Marine Fish,Achiridae,Achirus mucuri,Halibut,Vulnerable (VU),VU,Extracao Direta:Caça/Pesca; Outras Atividades ...,Yes,No,No,1.0,Yes,BA,BA,Wildfires,Atlantic Forest
3,Fauna,Aquatic Invertebrates,Aeglidae,Aegla brevipalma,Egla,Critically Endangered (CR),CR,Agropecuária; Poluição,No,Yes,No,1.0,Yes,SC,SC,Fishing,Atlantic Forest
4,Fauna,Aquatic Invertebrates,Aeglidae,Aegla camargoi,Egla,Endangered (EN),EN,Agropecuária; Outras Atividades Econômicas:Ene...,No,Yes,No,1.0,Yes,RS; SC,RS,Fishing,Atlantic Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Fauna,Freshwater Fish,Loricariidae,Hypancistrus zebra,True Grouper,Critically Endangered (CR),CR,Extracao Direta:Caça/Pesca; Outras Atividades ...,No,Yes,Yes,2.0,Yes,PA,PA,Human Settlement,Amazon
1074,Fauna,Freshwater Fish,Loricariidae,Hypancistrus zebra,True Grouper,Critically Endangered (CR),CR,Extracao Direta:Caça/Pesca; Outras Atividades ...,No,Yes,Yes,2.0,Yes,PA,PA,Mining,Amazon
1075,Fauna,Freshwater Fish,Loricariidae,Hypancistrus zebra,True Grouper,Critically Endangered (CR),CR,Extracao Direta:Caça/Pesca; Outras Atividades ...,No,Yes,Yes,2.0,Yes,PA,PA,Pollution,Amazon
1076,Fauna,Freshwater Fish,Characidae,Hyphessobrycon coelestinus,Unknown,Endangered (EN),EN,Assentamento Humano:cidades,Yes,Yes,Yes,2.0,Yes,DF; GO,DF,Fishing,Cerrado


In [15]:
df = df.drop(columns= ['occurrence_states_x', 'main_threats_x', 'fauna'])

In [16]:
categories = {
    'Sharks': [
        'Alopiidae',
        'Bouchardiidae',
        'Carcharhinidae',
        'Odontaspididae',
        'Lamnidae',
        'Cetorhinidae',
        'Triakidae',
        'Ginglymostomatidae',
    ],
    'Fish': [
        'Pimelodidae',
        'Rivulidae',
        'Characidae',
        'Apteronotidae',
        'Crenuchidae',
        'Heptapteridae',
        'Serranidae',
        'Poeciliidae',
        'Cichlidae',
        'Gobiidae',
        'Tripterygiidae',
        'Epinephelidae',
        'Doradidae',
        'Labridae',
    ],
    'Arthropods': [
        'Aeglidae',
        'Gecarcinidae',
    ],
    'Starfish': [
        'Astropectinidae',
        'Asteriidae',
    ],
    'Rays': [
        'Arhynchobatidae',
        'Gymnuridae',
    ],
    'Electric Fish': [
        'Hypopomidae',
        'Sternopygidae',
    ],
    'Freshwater Fish': [
        'Hyriidae',
        'Loricariidae',
        'Parodontidae',
        'Actiniidae',
        'Metaniidae',
        'Callichthyidae',
        'Cassidulidae',
        'Dasyatidae',
        'Onuphidae',
        'Trichomycteridae',
        'Eunicidae',
    ],
    'Seahorses': [
        'Syngnathidae',
    ],
    'Invertebrates and Crustaceans': [
        'Achiridae',
        'Ginglymostomatidae',
        'Halichondriidae',
        'Pectinidae',
    ],
}

# Function to map species to categories
def map_species_to_category(species):
    for category, species_list in categories.items():
        if species in species_list:
            return category
    return 'Other'  # Default category if not found

# Map the categories to the DataFrame based on the species names
df['category'] = df['family'].apply(map_species_to_category)
df['category'].value_counts(normalize=True)

category
Fish                             0.467532
Sharks                           0.179963
Freshwater Fish                  0.124304
Arthropods                       0.089054
Starfish                         0.054731
Seahorses                        0.034323
Invertebrates and Crustaceans    0.017625
Rays                             0.014842
Other                            0.014842
Electric Fish                    0.002783
Name: proportion, dtype: float64

In [17]:
nome_comum = [x.split('-')[0] for x in df['commum_name'].unique()]

In [18]:
with open("data/brazil-states.geojson") as f:
    gj = geojson.load(f)
features = gj

In [19]:
coordinates = pd.json_normalize(pd.DataFrame(features)['features'])['geometry.coordinates'].explode().explode().explode()
lats_lons = [[x[0], x[1]] for x in coordinates]

In [20]:
df_lat_lon = pd.DataFrame(lats_lons)
df_lat_lon['index'] = coordinates.reset_index()['index']

In [21]:
df_features = pd.json_normalize(pd.DataFrame(features)['features']).reset_index()
df_features= df_features.merge(coordinates.reset_index(), on='index')
df_features[df_features['properties.sigla'] == 'AC']

,index,type,geometry.type,geometry.coordinates_x,properties.id,properties.name,properties.sigla,properties.regiao_id,properties.codigo_ibg,properties.cartodb_id,properties.created_at,properties.updated_at,geometry.coordinates_y
0,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-73.608178, -7.201942]"
1,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-72.86963, -7.528509]"
2,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-72.663035, -7.622899]"
3,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-72.660926, -7.623753]"
4,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-72.658816, -7.624607]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-73.796145, -7.114988]"
2476,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-73.763846, -7.131794]"
2477,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-73.753143, -7.137184]"
2478,0,Feature,MultiPolygon,"[[[[-73.608178, -7.201942], [-72.86963, -7.528...",1,Acre,AC,3,12,1,2015-02-09T16:46:01Z,2015-02-09T16:46:01Z,"[-73.742441, -7.142574]"


In [22]:
region_animals = df['occurrence_states_y'].tolist()

In [23]:
lats_and_longs_animals = []

for region in region_animals:
    try:
        if region in ['BA', 'SE', 'PE', 'AL', 'RN', 'CE', 'MA', 'PI', 'AP', 'PA', 'RJ', 'SP', 'ES', 'SC', 'RS']:
            lat_lon = df_features[df_features['properties.sigla'] == region].sample(1)['geometry.coordinates_y'].values[0] + (np.random.random((1, 2))[0] / 10)
        else:
            k = 'default_value'  # Substitua 'default_value' pelo valor desejado para estados que não estão na lista acima
            lat_lon = df_features[df_features['properties.sigla'] == region].sample(1)['geometry.coordinates_y'].values[0] + (np.random.random((1, 2))[0] / 1)
        
        lats_and_longs_animals.append(lat_lon)
    except:
        lats_and_longs_animals.append([0, 0])


In [24]:
df['lats_longs_animals'] = lats_and_longs_animals


In [25]:
columns_to_binary =['presence_in_protected_areas', 'national_action_plan_for_conservation_',
                    'fishing_planning', 'exclusive_species_of_brazil']
binary_values_to_replace = {'Yes':True,
                            'No':False}

for i in columns_to_binary:
    df[i] = df[i].replace(binary_values_to_replace)
    
df['acronym_threat_category'] = df['acronym_threat_category'].replace({'CR/PEX':'PEX'})
df['threat_category'] = df['threat_category'].replace({'Critically Endangered (CR)/Possibly Extinct (PEX)':'Possibly Extinct (PEX)'})

In [26]:
df.to_csv('data/df_translated.csv', index=False)